In [10]:
import os
import sys
sys.path.append(os.path.realpath('../..'))
import aoc
my_aoc = aoc.AdventOfCode(2018,17)

In [11]:
input_text = """x=495, y=2..7
y=7, x=495..501
x=501, y=3..7
x=498, y=2..4
x=506, y=1..2
x=498, y=10..13
x=504, y=10..13
y=13, x=498..504"""
input_lines = input_text.splitlines()


In [12]:
import re
from grid import Grid
my_grid = Grid([])

input_pattern = re.compile(r'([xy])=(\d+), ([xy])=(\d+)..(\d+)')
for line in input_lines:
    match = input_pattern.match(line)
    if match:
        if match.group(1) == 'x':
            pos_x = int(match.group(2))
            for pos_y in range(int(match.group(4)), int(match.group(5)) + 1):
                my_grid.set_point((pos_x, pos_y), "#")
        else:
            pos_y = int(match.group(2))
            for pos_x in range(int(match.group(4)), int(match.group(5)) + 1):
                my_grid.set_point((pos_x, pos_y), "#")

my_grid.set_point((500, 0), "+")     
my_grid.update()


In [13]:
print(my_grid)

     +      
           #
#  #       #
#  #  #     
#  #  #     
#     #     
#     #     
#######     
            
            
   #     #  
   #     #  
   #     #  
   #######  


In [14]:
def expand_water(water, my_grid):
    water_list = list(water)
    for pos_x, pos_y in water_list:
        if (pos_x, pos_y) == (499,9):
            print(f"X: {my_grid.get_point((pos_x, pos_y), '.', '%')}")
        if my_grid.get_point((pos_x, pos_y), '.', '%') in ['+', '|']:
            # down sand
            if (pos_x, pos_y) == (499,9):
                print(f"D: {my_grid.get_point((pos_x, pos_y + 1), '.', '%')}")
            if my_grid.get_point((pos_x, pos_y + 1), '.', '%') in ['.']:
                water.add((pos_x, pos_y + 1))
                my_grid.set_point((pos_x, pos_y + 1), '|')
                return expand_water(water, my_grid)
            # down water
            elif my_grid.get_point((pos_x, pos_y + 1), '.', '%') in ['|']:
                pass
            # down clay or resting
            elif my_grid.get_point((pos_x, pos_y + 1), '.', '%') in ['#', '~']:
                if (pos_x, pos_y) == (499,9):
                    print(f"L: {my_grid.get_point((pos_x - 1 , pos_y), '.', '%')}")
                    print(f"R: {my_grid.get_point((pos_x + 1 , pos_y), '.', '%')}")
                # left sand
                if my_grid.get_point((pos_x - 1 , pos_y), '.', '%') in ['.']:
                    water.add((pos_x - 1 , pos_y))
                    my_grid.set_point((pos_x - 1, pos_y), '|')
                    return expand_water(water, my_grid)
                # right sand
                elif my_grid.get_point((pos_x + 1 , pos_y), '.', '%') in ['.']:
                    water.add((pos_x + 1 , pos_y))
                    my_grid.set_point((pos_x + 1, pos_y), '|')
                    return expand_water(water, my_grid)
                # left clay or resting
                #elif my_grid.get_point((pos_x - 1 , pos_y), '.', '%') in ['#', '~']:
                #    pass
                    # my_grid.set_point((pos_x, pos_y), '~')
                    # water, my_grid = expand_water(water, my_grid)
                # right clay or resting
                elif my_grid.get_point((pos_x + 1 , pos_y), '.', '%') in ['#', '~']:
                    my_grid.set_point((pos_x, pos_y), '~')
                    return expand_water(water, my_grid)

    return water, my_grid

water = set([(500,0)])
water, my_grid = expand_water(water, my_grid)
water, my_grid = expand_water(water, my_grid)

#print(water)
#print(my_grid)


X: |
D: |
X: |
D: ~
L: .
R: |
X: |
D: ~
L: |
R: |
X: |
D: ~
L: |
R: |
X: |
D: ~
L: |
R: |
X: |
D: ~
L: |
R: |
X: |
D: ~
L: |
R: |


In [15]:
total = 0
my_str = str(my_grid)
total += my_str.count('|')
total += my_str.count('~')
print(total)
print(len(water), water)


57
58 {(502, 8), (502, 11), (505, 10), (497, 6), (499, 3), (505, 13), (497, 12), (497, 9), (499, 6), (499, 12), (499, 9), (501, 12), (503, 9), (501, 9), (503, 12), (500, 1), (500, 4), (500, 10), (502, 7), (502, 4), (502, 10), (497, 5), (505, 12), (505, 9), (499, 5), (496, 6), (501, 2), (499, 2), (498, 9), (498, 6), (497, 11), (499, 11), (501, 11), (503, 11), (500, 0), (500, 3), (500, 6), (502, 3), (502, 9), (500, 12), (500, 9), (502, 6), (502, 12), (496, 5), (505, 11), (497, 10), (498, 5), (499, 4), (499, 10), (497, 13), (501, 10), (503, 10), (504, 9), (500, 5), (502, 2), (500, 2), (502, 5), (500, 11)}


In [16]:
my_grid.get_point((499,9))

'|'

In [72]:
from heapq import heappop, heappush
from collections import deque

def is_full(point, my_grid):
    # start at point
    (pos_x, pos_y) = point
    symbol = my_grid.get_point((pos_x, pos_y))
    # move left
    while symbol != '#':
        pos_x -= 1
        symbol = my_grid.get_point((pos_x, pos_y))
        if symbol not in ['#', '|', '~']:
            return False
    (pos_x, pos_y) = point
    symbol = my_grid.get_point((pos_x, pos_y))
    # move right
    while symbol != '#':
        pos_x += 1
        symbol = my_grid.get_point((pos_x, pos_y))
        if symbol not in ['#', '|', '~']:
            return False
    return True

class Heap():
    def __init__(self, heap=[]):
        self.heap = deque(heap)
    
    def popleft(self):
        return self.heap.popleft()
    
    def append(self, item):
        if item not in self.heap:
            self.heap.append(item)
    
def expand_water(start_point, my_grid):
    """
    Function to expand water based on provided rules
    """

    heap = Heap()
    (pos_x, pos_y) = start_point
    # start point is the water source, so lets not even consider it
    my_grid.set_point((pos_x, pos_y + 1), '|')
    heap.append((pos_x, pos_y + 1))
    counter = 0
    while heap:
        counter += 1
        if counter > 3000:
            print("breaking")
            print(heap)
            return my_grid
        (pos_x, pos_y)  = heap.popleft()
        points = {
            "current": (pos_x, pos_y),
            "left": (pos_x - 1, pos_y),
            "right": (pos_x + 1, pos_y),
            "up": (pos_x, pos_y - 1),
            "down": (pos_x, pos_y + 1)
        }
        symbols = {}
        for key, point in points.items():
            symbols[key] = my_grid.get_point(point)
        #print(counter, score, points, symbols)
        opposite = {
            'left': 'right',
            'right': 'left'
        }
        # conditions
        # . down
        if symbols['down'] == '.':
            my_grid.set_point( points['down'], '|')
            heap.append(points['down'])
            continue
        # | below
        # I don't think we are going to get here
        # # below or ~ below
        
        if symbols['down'] in ['#', '~']:
            # check left and right
            for direction in ['left', 'right']:
                if symbols[direction] == '.':
                    my_grid.set_point(points[direction], '|')
                    heap.append(points[direction])
                elif symbols[direction] == '|':
                    heap.append(points[direction])
                elif symbols[direction] in  ['#', '~']:
                    if is_full(points['current'], my_grid):
                        my_grid.set_point(points['current'], '~')
                        if symbols['up'] == '|':
                            heap.append(points['up'])
                    else:
                        heap.append(points['current'])
                    if symbols[opposite[direction]] not in ['#', '~']:
                        heap.append(points[opposite[direction]])
        else:
            heap.append(points['down'])
    return my_grid



In [73]:
import re
from grid import Grid
my_grid = Grid([])

input_pattern = re.compile(r'([xy])=(\d+), ([xy])=(\d+)..(\d+)')
for line in input_lines:
    match = input_pattern.match(line)
    if match:
        if match.group(1) == 'x':
            pos_x = int(match.group(2))
            for pos_y in range(int(match.group(4)), int(match.group(5)) + 1):
                my_grid.set_point((pos_x, pos_y), "#")
        else:
            pos_y = int(match.group(2))
            for pos_x in range(int(match.group(4)), int(match.group(5)) + 1):
                my_grid.set_point((pos_x, pos_y), "#")
my_grid.update()
my_grid.set_point((500, 0), "+")     
my_grid = expand_water((500, 0), my_grid)
print(my_grid)

breaking
     |     #
#  #||||   #
#  #~~#|    
#  #~~#|    
#~~~~~#|    
#~~~~~#|    
#######|    
       |    
  ||||||||| 
  |#~~~~~#| 
  |#~~~~~#| 
  |#~~~~~#| 
  |#######| 
